In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(0.25)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(0.25)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(0.25)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.25)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('HP_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-baddi-himachal-pradesh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6AE47B095+29557]
	(No symbol) [0x00007FF6AE3EFA50]
	(No symbol) [0x00007FF6AE2AB56A]
	(No symbol) [0x00007FF6AE2FF695]
	(No symbol) [0x00007FF6AE2FF8EC]
	(No symbol) [0x00007FF6AE34B777]
	(No symbol) [0x00007FF6AE3271CF]
	(No symbol) [0x00007FF6AE34851C]
	(No symbol) [0x00007FF6AE326F33]
	(No symbol) [0x00007FF6AE2F116F]
	(No symbol) [0x00007FF6AE2F22D1]
	GetHandleVerifier [0x00007FF6AE7AC96D+3378253]
	GetHandleVerifier [0x00007FF6AE7F8497+3688311]
	GetHandleVerifier [0x00007FF6AE7ED1CB+3642539]
	GetHandleVerifier [0x00007FF6AE53A6B6+813462]
	(No symbol) [0x00007FF6AE3FAB5F]
	(No symbol) [0x00007FF6AE3F6B74]
	(No symbol) [0x00007FF6AE3F6D10]
	(No symbol) [0x00007FF6AE3E5C1F]
	BaseThreadInitThunk [0x00007FFEC8F7257D+29]
	RtlUserThreadStart [0x00007FFECAEEAF28+40]

Error occurred while scraping bus details for https://www.redb

NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,Laxmi holidays,Scania Multi-Axle AC Semi Sleeper (2+2),23:30,08h 30m,08:00,4.5,1044,20 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,Bedi Travels,Volvo A/C Semi Sleeper (2+2),23:00,09h 00m,08:00,4.8,692,4 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,Zingbus Plus,Bharat Benz A/C Semi Sleeper (2+2),22:35,09h 35m,08:10,4.4,995,14 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,Laxmi holidays,Volvo A/C Semi Sleeper (2+2),23:25,08h 20m,07:45,4.4,1092,6 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,Ram Dalal Holidays,Mercedes Multi-Axle Semi Sleeper (2+2),23:30,08h 10m,07:40,4.5,891,7 Seats available
...,...,...,...,...,...,...,...,...,...,...
133,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...,Bedi Travels,Volvo A/C Semi Sleeper (2+2),23:15,07h 15m,06:30,5.0,585,16 Seats available
134,Chandigarh to Kangra,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,VE A/C Semi Sleeper (2+2),23:50,05h 00m,04:50,4.8,759,14 Seats available
135,Chandigarh to Kangra,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),23:55,05h 15m,05:10,4.5,949,5 Seats available
136,Chandigarh to Kangra,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,Scania Multi-Axle AC Semi Sleeper (2+2),23:55,05h 15m,05:10,4.4,997,6 Seats available
